# A simple Rag Application Using Python/Langchain/Ollama 

In [1]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf

import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Load PDF

In [3]:
from PyPDF2 import PdfReader
from langchain.schema import Document

# Load the PDF
local_path = "C:\\Karpagam-Bio-Batch1-Rag-app\\pdfs\\01.pdf"

if local_path:
    # Read PDF and extract text
    reader = PdfReader(local_path)
    data=[]

    for page in reader.pages:
        text = page.extract_text()
        if text:
            data.append(text)

    print(f"PDF loaded successfully: {local_path}")
    print(f"Extracted {len(data)} pages of content.")
else:
    print("Upload a PDF file")

PDF loaded successfully: C:\Karpagam-Bio-Batch1-Rag-app\pdfs\01.pdf
Extracted 2 pages of content.


## Split text into chunks

In [4]:
documents = [Document(page_content=page, metadata={"source": f"Page {i + 1}"}) for i, page in enumerate(data)]

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
print(f"Text split into {len(chunks)} chunks")

# Print a sample chunk for verification
for i, chunk in enumerate(chunks[:3]):  # Show only the first 3 chunks
    print(f"\nChunk {i + 1}:\n{chunk.page_content}")

Text split into 7 chunks

Chunk 1:
[ NationalRad Sample Body Radiology Report ]     Imaging Center 123 Main Street Anywhere, USA 01234 Phone 123.456.7890 Fax 123.456.7890   PATIENT: JOHN SMITH DOB: 5/5/1955 FILE #: 12345 PHYSICIAN: REFERRING EXAM: MRI ABDOMEN WITH CONTRAST DATE: 1/1/2011  
Report approved on    NationalRad | Headquartered: Florida | Diagnostic Imaging Services: Nationwide | 877.734.6674 | www.NationalRad.com

Chunk 2:
CLINICAL INFORMATION  History pancreatic cancer. Status post aortic chemotherapy and Whipple procedure on DATE. Chronic low back pain. Abdominal pain. Follow-up examination.  COMPARISON  Comparison is made with previous CT scan reported DATE and DATE  CONTRAST  15 mL of MultiHance was administered per protocol.  TECHNIQUE  Coronal T2-weighted axial and T2; axial T2 fat sat clear, T2 and, T2 gradient-echo, and in phase sequences; dynamic axial T1 fat-sat post contrast additional subtraction reconstructions; coronal single shot MRCP sequences  FINDINGS  Mar

## Create vector database

In [5]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


## Set up LLM and Retrieval

In [6]:
# Set up LLM and retrieval
local_model = "llama3.2:3b"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [12]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are a medical Assistant Bot you need to clearly analyse the data
    from the chunks retrived from the vector database and retrive chunks similar and 
    related to the question asked and also mainly tell if there are any abnormalities found 
    on the report and if any abnormality found please give a recommendataion to visit the
    doctor clinic. Please note this is the question asked by user on web browser screen : {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [13]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [14]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [15]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [17]:
# Example 1
chat_with_pdf("What is the main idea of this document?")

The main idea of this document appears to be a radiology report from an MRI examination, specifically of the abdomen with contrast, performed on January 1, 2011. The report highlights various findings, including marked hydronephrosis and hydroureter in the right kidney, stable mild pelviectasis in the left kidney, postoperative changes from previous Whipple procedure, and other abdominal abnormalities. The report concludes with recommendations for further evaluation, such as a CT scan of the abdomen and pelvis with and without contrast.

## Clean up (optional)

In [ ]:
# Optional: Clean up when done 
vector_db.delete_collection()
print("Vector database deleted successfully")

Vector database deleted successfully
